In [1]:
Dataset = "FoCus"                                  
LLM_name = "Qwen2-5B-Instruct-Full"                   

_COT = ""
# _COT = "-COT"

In [ ]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

# ds = pd.read_csv(f'New Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds


Shape: (1000, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,"Note: \n- You will not write actual responses,...",0.524928
1,"Note: You will not write actual responses, jus...",0.452668
2,"Note: You will not write actual responses, jus...",0.346046
3,"Note: You will not write actual responses, jus...",0.626562
4,Note: Provide a persona response if provided\n...,0.345477
...,...,...
995,"Note: You will not write User1, User2, etc. In...",0.862803
996,Note: The personalized response should reflect...,0.509085
997,Note: You will have multiple chat interactions...,0.841039
998,"Note: You will not write User2's name, so you ...",0.515458


In [3]:
print(ds.iloc[167]["gen_response"])

Note: You will not write actual responses, just personalized ones for the provided persona.
{
  "response": "Here's what I remember about Thursday Island Cemetery: It's a cemetery where Japanese people used to bury their dead. Many of them were divers who found pearls in the area."
}


In [4]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    
    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     172
response_time      0
dtype: int64
(1000, 2)


,gen_response,response_time
0,The Nazareth House is a historic institution l...,0.524928
1,The Truman Galusha House is a historic house l...,0.452668
2,The Marion Palace Theatre is a movie palace in...,0.346046
3,Technische Universität Darmstadt is a research...,0.626562
4,Here's a suggestion for visiting England: Bost...,0.345477
...,...,...
995,None,0.862803
996,It sounds like you're planning a trip to Bangl...,0.509085
997,None,0.841039
998,Nyanga National Park is located in South Afric...,0.515458


In [5]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)